# Сокращение размерности

### EM алгоритм

Напомним, как выглядит $EM$ алгоритм. 

$$
\log p(x|\theta) = \int q(z)\log p(x|\theta)dz = \int q(z)\log\dfrac{p(x,z|\theta)}{p(z|x,\theta)}dz = \int q(z)\log\dfrac{p(x,z|\theta)}{q(z)}dz+\int q(z)\log\dfrac{q(z)}{p(z|x,\theta)}dz = \mathcal{L}(q,\theta)+KL(q(z)|p(z|x,\theta))
$$

** E-step ** 
$$ q(z)^{(n+1)} = p(z|x,\theta^{(n)}) $$
** M-step ** 
$$ \theta^{(n+1)} = \max\limits_{\theta} \mathcal{L}(q^{(n+1)},\theta) $$

### PCA как модель с латентными пременными

Процесс генерации данных в модели PCA выглядит следующим образом:

![Генеративный процесс](pca_illustration.png "Генеративный процесс")

Таким образом, мы получаем модель:

\begin{equation}P(X,T|W,\mu,\sigma^2) = \prod\limits_{n=1}^{N}p(x_n,t_n|W,\mu,\sigma^2) = \prod\limits_{n=1}^{N}\mathcal{N}(x_n|Wt_n+\mu,\sigma^2I)\mathcal{N}(t_n|0,I)
\end{equation}

** Задание 1. ** (5 баллов) Выпишите E и M шаги для модели PCA.

$\mathcal{L}_C$ - complete-data log-likelihood function         
$\langle \cdot \rangle$ - expectation           
d - data space dimension             
S - sample covariance matrix             
T - error accounted data matrix            
$1_N$ - vector with N ones             

![](2.png)

For the complete data T and X, the above joint probability distribution can be written as

![](3.png)

and the complete-data log-likelihood function $\mathcal{L}_C$ is

![](4.png)

In order for the EM-PCA to estimate $x_j$ in the E-step, the posterior distribution of $x_j$ given $t_j$ is calculated using Bayes’ rule.

![](5.png)

with

![](6.png)

which is a Gaussian distribution such that

![](7.png)

Note that a matrix identity $W^T (W W^T + \sigma^2 I)^{−1} = (W^T W + \sigma^2 I)^{−1} W^T$ is utilized in the above derivation to save computational costs for matrix inversion (*).  For the M-step, the expectation of $\mathcal{L}_C$ is found as follows.

![](8.png)

where

![](9.png)

![](10.png)

By requiring the derivative of $\langle \mathcal{L}_C \rangle$ with respect to each parameter to vanish, the parameter estimators turn out as

![](11.png)

![](12.png)

Finally, the EM-PCA algorithm reduces to the following form by limiting $\sigma^2$ to infinitesimal for a zero noise case.

![](13.png)

Proof for (*) in 14.png - 16.png

** Задание 2. ** (5 баллов) Реализуйте EM алгоритм для модели PCA

In [5]:
import numpy as np
from numpy.linalg import inv
from numpy import transpose as tr


class PCA(object):
    def __init__(self, q=2, sigma=1.0):
        self.q = q
        self.prior_sigma = sigma

    def fit(self, y):
        self.y = y
        self.p = y.shape[0]
        self.n = y.shape[1]
        [self.w, self.mu, self.sigma] = self.__fit_em()

    def __ell(self, w, mu, sigma, norm=True):
        m = inv(tr(w).dot(w) + sigma * np.eye(w.shape[1]))
        mw = m.dot(tr(w))
        ll = 0.0
        for i in range(self.n):
            yi = self.y[:, i][:, np.newaxis]
            yyi = yi - mu
            xi = mw.dot(yyi)
            xxi = sigma * m + xi.dot(tr(xi))
            ll += 0.5 * np.trace(xxi)
            if sigma > 1e-5:
                ll += (2 * sigma)**(-1) * float(tr(yyi).dot(yyi))
                ll -= sigma**(-1) * float(tr(xi).dot(tr(w)).dot(yyi))
                ll += (2 * sigma)**(-1) * np.trace(tr(w).dot(w).dot(xxi))
        if sigma > 1e-5:
            ll += 0.5 * self.n * self.p * np.log(sigma)
        ll *= -1.0
        if norm:
            ll /= float(self.n)
        return ll

    def __fit_em(self, max_it=100):
        w = np.random.rand(self.p, self.q)
        mu = np.mean(self.y, 1)[:, np.newaxis]
        sigma = self.prior_sigma
        ll = self.__ell(w, mu, sigma)

        yy = self.y - mu
        s = self.n**(-1) * yy.dot(tr(yy))
        for i in range(max_it):
            m = inv(tr(w).dot(w) + sigma * np.eye(self.q))
            t = inv(sigma * np.eye(self.q) + m.dot(tr(w)).dot(s).dot(w))
            w_new = s.dot(w).dot(t)
            sigma_new = self.p**(-1) * np.trace(s - s.dot(w).dot(m).dot(tr(w_new)))
            ll_new = self.__ell(w_new, mu, sigma_new)
            w = w_new
            sigma = sigma_new
            ll = ll_new
        return (w, mu, sigma)

In [6]:
model = PCA()

In [7]:
y = np.random.rand(10, 5)

In [12]:
y

array([[ 0.64786705,  0.9244519 ,  0.15194038,  0.31842225,  0.88663749],
       [ 0.35710602,  0.08625129,  0.22778666,  0.29820592,  0.43705772],
       [ 0.25158284,  0.03052652,  0.94938675,  0.14545179,  0.92980012],
       [ 0.47828738,  0.65129402,  0.70422171,  0.14786224,  0.2218021 ],
       [ 0.30494854,  0.14177862,  0.02826922,  0.29980284,  0.61033674],
       [ 0.53636041,  0.59538634,  0.93660967,  0.04702375,  0.42295589],
       [ 0.49236315,  0.3253516 ,  0.08010441,  0.80670182,  0.75630276],
       [ 0.8982511 ,  0.30693702,  0.06304511,  0.07601901,  0.39971064],
       [ 0.52445366,  0.79688895,  0.49692206,  0.49483858,  0.74506941],
       [ 0.31650554,  0.94042846,  0.04795046,  0.73016825,  0.42204081]])

In [8]:
model.fit(y)

In [9]:
model.w

array([[ 0.08893495, -0.13410495],
       [ 0.08627561,  0.05583572],
       [ 0.01329896,  0.35736019],
       [-0.19862696,  0.01506801],
       [ 0.17041287,  0.01434414],
       [-0.21285162,  0.13133486],
       [ 0.24365406, -0.06374657],
       [ 0.0524461 , -0.04515896],
       [ 0.013977  , -0.04432729],
       [ 0.05775522, -0.27029147]])

In [10]:
model.mu

array([[ 0.58586382],
       [ 0.28128152],
       [ 0.4613496 ],
       [ 0.44069349],
       [ 0.27702719],
       [ 0.50766721],
       [ 0.49216475],
       [ 0.34879258],
       [ 0.61163453],
       [ 0.49141871]])

In [11]:
model.sigma

0.027463765907009614

** Задание 3. ** (10 баллов) Используя свою реализацию метода PCA, решите задачу восстанавления давления по профилю крыла.

Скачать данные со страницы курса. https://yadi.sk/i/065iNGXp3QYv5n

Данные представляют собой матрицу размера $397\times114$; в каждой
строке находится подвектор, описывающий профиль (первые 57
координат), и подвектор, описывающий соответствующее распределение
давления (следующие 57 координат). Задача состоит в том, чтобы на
основании метода главных компонент построить алгоритм, позволяющий
восстанавливать распределение давления по профилю. Опишем алгоритм
обучения и алгоритм восстановления. Пусть ${\bf A}\in\mathbb{R}^{57}$ --
вектор, описывающий профиль, а ${\bf P}\in\mathbb{R}^{57}$ -- вектор,
описывающий распределение давления. Дана обучающая выборка $({\bf
A}_i,{\bf P}_i)_{i=1}^N$ (взять в качестве обучающей выборки 75%
случайно выбранных строк из матрицы данных; остальные 25%
использовать в качестве тестового множества данных).

Алгоритм обучения состоит из следующих шагов:


1. По данным $({\bf A}_i,{\bf P}_i)_{i=1}^N$ оцениваются первые
$d$ (параметр алгоритма) главных компонент ${\bf e}_1,\ldots,{\bf
e}_d$, где ${\bf e}_i\in\mathbb{R}^{114}$. 
2. Каждый из векторов ${\bf e}_i,i=1,\ldots,d$ представляется как объединение двух подвекторов
${\bf e}_i = ({\bf e}_i^A\in\mathbb{R}^{57},{\bf
e}_i^P\in\mathbb{R}^{57}),i=1,\ldots,d$, соответствующих описанию профиля
и распределению давления соответственно.


Преобразование произвольного объединенного вектора ${\bf Z} =
({\bf A},{\bf P})$ в сжатое описание ${\bf\lambda}=(\lambda_1,\ldots,\lambda_d)$ происходит согласно
следующей формуле:

$$
\lambda_i = \left( {\bf Z}-{\bf Z}_{\mathrm{mean}},{\bf e}_i\right),
i=1,\ldots,d,
$$

где ${\bf Z}_{\mathrm{mean}} = \frac{1}{N}\sum_{k=1}^N {\bf Z}_k$,
${\bf Z}_k = ({\bf A}_k,{\bf P}_k)$ (выборочное среднее;
подсчитывается по обучающей выборке). Восстановления подвекторов
${\bf A}$ и ${\bf P}$ объединенного вектора ${\bf Z} = ({\bf
A},{\bf P})$ по сжатому описанию ${\bf
\lambda}=(\lambda_1,\ldots,\lambda_d)$ происходит согласно
формулам

$$
{\bf A}^*({\bf \lambda}) = {\bf A}_{\mathrm{mean}} + \sum_{i=1}^d\lambda_i{\bf e}_i^A,\,{\bf
P}^*({\bf \lambda}) ={\bf P}_{\mathrm{mean}} + \sum_{i=1}^d\lambda_i{\bf e}_i^P.
$$


Алгоритм восстановления давления ${\bf P}$ по профилю ${\bf A}$
может быть описан следующим образом:


1. Для заданного профиля ${\bf A}$ определяются такие ${\bf
\lambda}=(\lambda_1,\ldots,\lambda_d)$, что $\|{\bf A}^*({\bf
\lambda})-{\bf A}\|_{2}^2$ принимает минимальное значение (такое
значение ${\bf \lambda}$ подсчитывается с помощью псевдообращения
матрицы $E=[{\bf e}_1^A,\ldots,{\bf e}_d^A]$, то есть ${\bf
\lambda} = E^{+}\left({\bf A}-{\bf A}_{\mathrm{mean}}\right)$, где $E^+$ -- псевдообратная матрица).
2. По вычисленному вектору ${\bf\lambda}$ оценивается давление согласно формуле $[{\bf P}^*({\bf
\lambda}) = {\bf P}_{\mathrm{mean}} + \sum_{i=1}^d\lambda_i{\bf e}_i^P.
]$

Задача состоит в том, чтобы реализовать предложенный алгоритм;
подсчитать график зависимости средней ошибки восстановления
векторов давлений из тестовой выборки от размерности сжатия $d$;
нарисовать несколько восстановленных графиков давления (для
нескольких разных профилей) с наложением истинных значений
давления. Ошибка между набором векторов $({\bf P}_i)_{i=1}^M$ и
восстановленными векторами $({\bf P}^*_i)_{i=1}^M$ оценивается
согласно формуле $\sqrt{\frac{1}{M}\sum_{i=1}^M\|{\bf P}_i-{\bf P}^*_i\|^2}$, где
$\|\cdot\|$ -- евклидово расстояние

In [26]:
import pandas as pd

In [27]:
data = pd.read_csv('A2P.txt', sep=" ", header=None)
data = data.drop(114, axis=1)

In [28]:
data

,0,1,2,3,4,5,6,7,8,9,...,104,105,106,107,108,109,110,111,112,113
0,0.000000,0.000334,0.000841,0.001942,0.004899,0.008544,0.012635,0.017008,0.021522,0.026042,...,-0.010036,0.019890,0.050947,0.082768,0.112232,0.140162,0.166887,0.187999,0.194079,0.189350
1,0.000000,0.000334,0.000841,0.001942,0.004899,0.008544,0.012635,0.017008,0.021522,0.026042,...,-0.010036,0.019890,0.050947,0.082768,0.112232,0.140162,0.166887,0.187999,0.194079,0.189350
2,0.000000,0.002407,0.005409,0.010245,0.018965,0.026769,0.033867,0.040345,0.046239,0.051562,...,-0.036864,-0.005580,0.046540,0.111951,0.180465,0.242534,0.292701,0.345705,0.391550,0.456239
3,0.000000,0.002407,0.005409,0.010245,0.018965,0.026769,0.033867,0.040345,0.046239,0.051562,...,-0.036864,-0.005580,0.046540,0.111951,0.180465,0.242534,0.292701,0.345705,0.391550,0.456239
4,0.001114,0.002846,0.005096,0.008957,0.016417,0.023352,0.029759,0.035685,0.041163,0.046200,...,-0.046082,-0.037372,-0.027537,-0.015805,-0.000199,0.017445,0.048162,0.107345,0.168383,0.204293
5,0.001114,0.002841,0.004493,0.006953,0.011180,0.014750,0.017869,0.020643,0.023121,0.025331,...,-0.113928,-0.094690,-0.072045,-0.046364,-0.014929,0.024972,0.082737,0.175643,0.256436,0.310923
6,0.000744,0.002249,0.004082,0.007233,0.013736,0.020442,0.027285,0.034147,0.040727,0.046769,...,-0.098385,-0.054704,-0.011520,0.028541,0.062915,0.097268,0.138813,0.189867,0.230086,0.254145
7,0.000354,0.002697,0.005643,0.010460,0.019280,0.027017,0.033733,0.039455,0.044218,0.048180,...,-0.042670,-0.002797,0.034873,0.066585,0.096291,0.127435,0.163446,0.219981,0.272217,0.317904
8,0.000676,0.003163,0.006414,0.011798,0.021707,0.030256,0.037431,0.043226,0.047669,0.050968,...,-0.054082,0.039217,0.138188,0.236241,0.325948,0.389793,0.419796,0.423067,0.414952,0.425029
9,0.000676,-0.000638,-0.001846,-0.003154,-0.003398,-0.000861,0.004091,0.010620,0.017741,0.024735,...,-0.255697,-0.284230,-0.306459,-0.313848,-0.287458,-0.229917,-0.147257,-0.019377,0.077667,0.111736
